In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

In [65]:
data = pd.read_csv('input/train.csv')
# data = pd.read_csv('input/train_min.csv')  # small data

X, y = data.iloc[:,2:].values, data.iloc[:,1].values

# std scaling
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [82]:
#cv: 0.8597059678850029
# logistic = SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)

#cv: 0.8543658603036868
nn = MLPClassifier(solver='lbfgs', alpha=0.001,
                    hidden_layer_sizes=(5, 2), random_state=1)

cls = nn

# try grid-search
#params = { 'alpha': [ 0.03, 0.1,0.3,1,3 ]  }
#cls = GridSearchCV(estimator=logistic, param_grid=params, cv=5, scoring='roc_auc')
#cls.fit(X_scaled, y)
#print(cls.best_params_)
#print(cls.best_score_)

# single
cv_scores = cross_val_score(cls, X_scaled, y, cv=5, scoring='roc_auc')
print(cv_scores)
print(np.average(cv_scores))


[0.85100361 0.85245629 0.85354592 0.85711782 0.85770566]
0.8543658603036868


In [69]:
# create final output
test = pd.read_csv('input/test.csv')
test_ids, test_x = test.iloc[:,0], test.iloc[:,1:]

test_x_scaled = scaler.transform(test_x)
test_y = cls.predict_proba(test_x_scaled)

In [70]:
# output the result
sub_ids = pd.DataFrame(test_ids)
sub_y = pd.DataFrame(pd.DataFrame(test_y).iloc[:,1])

sub = pd.DataFrame(np.hstack((sub_ids, sub_y)))
sub.to_csv('output.csv', header=['ID_code', 'target'], index=False)

output_check = pd.read_csv('output.csv')
output_check.head()

,ID_code,target
0,test_0,0.186199
1,test_1,0.233491
2,test_2,0.057496
3,test_3,0.196701
4,test_4,0.073267
